In [1]:
import pathlib
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential

In [2]:
    # dataset path
    pic_path = pathlib.Path(r"C:\Users\LINDLARZ\Downloads\dataset")

In [3]:
# check folders
list(pic_path.iterdir())

[WindowsPath('C:/Users/LINDLARZ/Downloads/dataset/mammooty'),
 WindowsPath('C:/Users/LINDLARZ/Downloads/dataset/mohanlal')]

In [4]:
import pathlib

pic_path = pathlib.Path(r"C:\Users\LINDLARZ\Downloads\dataset")

mammootty_list = list(pic_path.glob("mammooty/**/*"))
mohanlal_list = list(pic_path.glob("mohanlal/**/*"))

print(f"Mammootty images: {len(mammootty_list)}")
print(f"Mohanlal images: {len(mohanlal_list)}")

if len(mammootty_list) > 0:
    print("Sample Mammootty:", mammootty_list[0])
else:
    print("No Mammootty images found.")

if len(mohanlal_list) > 0:
    print("Sample Mohanlal:", mohanlal_list[0])
else:
    print("No Mohanlal images found.")

Mammootty images: 2295
Mohanlal images: 2763
Sample Mammootty: C:\Users\LINDLARZ\Downloads\dataset\mammooty\1.jpeg
Sample Mohanlal: C:\Users\LINDLARZ\Downloads\dataset\mohanlal\0.jpeg


In [5]:
mammootty_list = list(pic_path.glob("mammooty/**/*"))
mohanlal_list = list(pic_path.glob("mohanlal/**/*"))

pic_dict = {'mammootty': mammootty_list, 'mohanlal': mohanlal_list}
pic_class = {'mammootty': 0, 'mohanlal': 1}

# Optional: print some paths
for i in pic_dict:
    print(f"\n{i}:")
    for j in pic_dict[i][:3]:  # print first 3 images from each class
        print(j)


mammootty:
C:\Users\LINDLARZ\Downloads\dataset\mammooty\1.jpeg
C:\Users\LINDLARZ\Downloads\dataset\mammooty\10.jpeg
C:\Users\LINDLARZ\Downloads\dataset\mammooty\100.jpeg

mohanlal:
C:\Users\LINDLARZ\Downloads\dataset\mohanlal\0.jpeg
C:\Users\LINDLARZ\Downloads\dataset\mohanlal\1.jpeg
C:\Users\LINDLARZ\Downloads\dataset\mohanlal\10.jpeg


In [6]:
x, y = [], []

for label in pic_dict:
    for img_path in pic_dict[label]:
        img = cv2.imread(str(img_path))

        if img is None:
            print("Skipping unreadable image:", img_path)
            continue

        img = cv2.resize(img, (100, 100))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0

        x.append(img)
        y.append(pic_class[label])

x = np.array(x)
y = np.array(y)

print(x.shape, y.shape)

(5058, 100, 100, 3) (5058,)


In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, train_size=0.8, random_state=1)

In [8]:
model = Sequential([
    Conv2D(50, (3,3), activation='relu', input_shape=(100,100,3), strides=1, padding='valid'),
    MaxPool2D(pool_size=(2,3)),
    Conv2D(100, (3,3), activation='relu', strides=1, padding='valid'),
    MaxPool2D(pool_size=(2,2)),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(), 
              loss=SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

history = model.fit(xtrain, ytrain, 
                    batch_size=32, epochs=50, 
                    validation_data=(xtest, ytest))

# save the model
model.save('face_model.h5')

D:\anaconda\envs\cnnenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 97s 716ms/step - accuracy: 0.5494 - loss: 0.7038 - val_accuracy: 0.5711 - val_loss: 0.6822
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 111s 874ms/step - accuracy: 0.6130 - loss: 0.6549 - val_accuracy: 0.6146 - val_loss: 0.6527
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 103s 814ms/step - accuracy: 0.7017 - loss: 0.5625 - val_accuracy: 0.6779 - val_loss: 0.6359
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 105s 827ms/step - accuracy: 0.8957 - loss: 0.2712 - val_accuracy: 0.7164 - val_loss: 0.7003
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 79s 621ms/step - accuracy: 0.9474 - loss: 0.1582 - val_accuracy: 0.7381 - val_loss: 0.7406
Epoch 7/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 82s 648ms/step - accuracy: 0.9763 - loss: 0.0893 - val_accuracy: 0.7470 - val_loss: 0.8289
Epoch 8/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 158s 770ms/step - accuracy: 0.9842 - loss: 0.0696 - val_accuracy: 0.7391 - val_loss: 0.9714
Epoch 9/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 93s 735ms/step - accuracy: 0.9874 - los

In [9]:
# Load model
model = tf.keras.models.load_model('face_model.h5')

# Labels
labels = {0: 'Mammootty', 1: 'Mohanlal'}

In [10]:
def detect_faces(frame, model, labels):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        cv2.putText(frame, 'No face detected', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 
                    1, (0,0,255), 2, cv2.LINE_AA)
        return frame

    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_resized = cv2.resize(face_rgb, (100,100))
        face_resized = np.expand_dims(face_resized/255.0, axis=0)
        
        pred = model.predict(face_resized)
        label = labels[pred.argmax()]
        
        cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.9, (0,255,0), 2)
    return frame

In [13]:
test_image_path = r'C:\Users\LINDLARZ\Downloads\mamoty1.jpeg'  # change this to your test image
frame = cv2.imread(test_image_path)

if frame is None:
    print("Error: Unable to load image.")
else:
    output_image = detect_faces(frame, model, labels)
    cv2.imshow("Prediction", output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


In [16]:
import cv2
import numpy as np
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model("face_model.h5")

# Labels mapping
labels = {0: 'Mammootty', 1: 'Mohanlal'}

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Prediction function
def predict_face(face_img):
    face_img = cv2.resize(face_img, (100, 100))
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = face_img / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    preds = model.predict(face_img)
    return labels[np.argmax(preds)]

# Open webcam
cap = cv2.VideoCapture(0)  # use 0 for the default camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        label = predict_face(face_img)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("Live Face Recognition", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == 27:  # ESC key
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━

In [18]:
from tensorflow.keras.models import load_model

# Load saved model
model = load_model("face_model.h5")

# Evaluate again on your test data
loss, accuracy = model.evaluate(xtest, ytest)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.7490 - loss: 1.4054
Test Accuracy: 74.90%
